# Cough analysis 
Here cough analysis is done through mfcc of the sound, this will convert our dataset into reduced numpy array which will be easy to train. Then we pass it through a RNN model that consist of 2 LSTM layer. We are currently planning to test its  accuracy with and with the embedding layer.
Embedding layer is basically a dense layer without biases. And it is added with the purpose to learn the relations between the words or any other format of data such that it becomes more easy of the recurrent network to learn the data making it act like a unsuperviesed layer of the network(I am not sure about this point) But any ways it is similar to the spatial transform netowrk layer in any convolutional neurl network


In [18]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.preprocessing.sequence as sequence
from tensorflow.keras.optimizers import Adagrad

import librosa as lb
import numpy as np
import matplotlib.pyplot as plt 
import os 
import json

import warnings 
warnings.filterwarnings("ignore")

# Preprocessing
In preprocessing we are using the library librosa and json to get sound and cough prediction 

In [19]:
data = r"C:\Users\suyash\Desktop\cough detection dataset"
files = os.listdir(data)
xtrain = []
ytrain = []
for c, file in enumerate(files):
    if file[-4:] == "json":
        file = open(data+"/"+file)
        txt = json.load(file)
        txt = txt['cough_detected']
        ytrain.append(txt)
    else:
        y, sr = lb.load(data+"/"+file)
        mfccs_features = lb.feature.mfcc(y=y, sr=sr, n_mfcc=16)
        xtrain.append(mfccs_features)
    if c>1998:
        break

# Normalization

In [20]:
for x in xtrain:
    shape = np.array(x).shape
    l = (shape[1] - shape[1]%4)/4
    new = x[:,:int(l*4)]
    # print(x.shape)
    new = np.reshape(new, (64, int(l)))
    new = sequence.pad_sequences(new, maxlen = 120, padding="post")
    new = np.reshape(new, (120, 64))
    xtrain[xtrain.index(x)] = new

In [21]:
ytrain = np.array([np.array(float(x)) for x in ytrain])

In [22]:
xtr = []
for x in xtrain:    
    xtr.append(new)
xtrain = np.array(xtr)

In [23]:
xtrain.shape

(1000, 120, 64)

In [24]:
# np.save("xtrain.npy", xtrain)
# np.save("ytrain.npy", ytrain)

# Model 


In [25]:
def RNN(inp):
    inputs = layers.Input(inp)
    lstm1 = layers.LSTM(64, activation="relu",return_sequences=True)(inputs)
    lstm2 = layers.LSTM(32, activation="relu", return_sequences=False)(lstm1)
    dense1 = layers.Dense(16, activation="relu")(lstm2)
    dense2 = layers.Dense(1, activation="relu")(dense1)

    model = models.Model(inputs=inputs, outputs=dense2, name="cough_detection")
    return model  

In [26]:
# model = models.Sequential([
#     layers.LSTM(64, activation="relu",return_sequences=True),
#     layers.LSTM(32, activation="relu", return_sequences=False),
#     layers.Dense(16, activation="relu"),
#     layers.Dense(1, activation="relu")
# ])

In [27]:
model = RNN(120, 64)
# model.summary()

In [34]:
model.summary()

Model: "cough_detection"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 120, 64)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 120, 64)           33024     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 45,985
Trainable params: 45,985
Non-trainable params: 0
_________________________________________________________________


# Training

In [29]:
from tensorflow.keras.optimizers import Adam

In [ ]:
model.compile(loss='MSE', optimizer=Adam(learning_rate=1e-7), metrics=['accuracy'])
model.fit(xtrain, ytrain, epochs=300, verbose=1, batch_size=20)

### Prediction

In [41]:
for x in range(100):  
    print(model.predict(xtr[x].reshape(1,120,64)), ytrain[x])

[[0.24522549]] 0.0155
[[0.7573408]] 0.9609
[[0.25434482]] 0.1643
[[0.78587335]] 0.9301
[[0.10959158]] 0.0482
[[0.76434195]] 0.9968
[[0.7829189]] 0.0735
[[0.23109436]] 0.0306
[[0.79022884]] 0.7811
[[0.14514093]] 0.0307
[[0.79339397]] 0.8937
[[0.7876431]] 0.9883
[[0.7927503]] 0.9456
[[0.79091614]] 0.9959
[[0.78934705]] 0.9536
[[0.7920141]] 0.9712
[[0.7469171]] 0.824
[[0.12001724]] 0.0576
[[0.3763579]] 0.3009
[[0.76733494]] 0.8109
[[0.08062581]] 0.0632
[[0.78584653]] 0.9882
[[0.7940916]] 0.9794
[[0.7838974]] 0.985
[[0.79065686]] 0.4277
[[0.6126307]] 0.6911
[[0.7966045]] 0.9664
[[0.774235]] 0.9947
[[0.7868097]] 0.9828
[[0.79879797]] 0.0456
[[0.79485226]] 0.0246
[[0.7908144]] 0.9472
[[0.7506182]] 0.9917
[[0.7868932]] 0.9854
[[0.79001224]] 0.9974
[[0.67672455]] 0.9794
[[0.11569441]] 0.3147
[[0.79042804]] 0.9948
[[0.78840315]] 0.9729
[[0.28359267]] 0.0088
[[0.7798016]] 0.5799
[[0.78633964]] 0.2677
[[0.7904539]] 0.9961
[[0.78314734]] 0.8904
[[0.5515471]] 0.2216
[[0.79239345]] 0.9734
[[0.781943

In [ ]:
for x in range(1000):  
    print(model.predict(xtr[x].reshape(1,120,64)), ytrain[x])

In [49]:
model.save("final1000.h5")

In [39]:
file = r"C:\Users\suyash\Desktop\KACHRA\laohub\Smile in Pain\Ajgar Ke Jalve\Artificiall Intelligence\Neural Networks\Supervised Learning\Recurrent Nets\RNN\Audio\final.h5"
model = models.load_model(file)